## Collecting protocol fees across Balancer core pools on all networks
- Spreadsheet as reference: https://docs.google.com/spreadsheets/d/1xwUPpbYq7woVOU9vQ8EB8MY75I-1mauTLyDVwvKUDKo/edit#gid=0
- Collab: https://colab.research.google.com/drive/1vKCvcV5mkL1zwW3565kLSGkBEbt8NsoB?usp=sharing


In [1]:
import datetime
import os

import pandas as pd
from dotenv import load_dotenv
from web3 import Web3
from web3.middleware import geth_poa_middleware

from notebooks import get_block_by_ts
from notebooks import get_twap_bpt_price
from notebooks.constants import ARB_CORE_POOLS
from notebooks.constants import BASE_CORE_POOLS
from notebooks.constants import GNOSIS_CORE_POOLS
from notebooks.constants import MAINNET_CORE_POOLS
from notebooks.constants import POLYGON_CORE_POOLS

load_dotenv()
arb_web3 = Web3(Web3.HTTPProvider(os.environ["ARBNODEURL"]))
eth_web3 = Web3(Web3.HTTPProvider(os.environ["ETHNODEURL"]))
base_web3 = Web3(Web3.HTTPProvider(os.environ["BASENODEURL"]))
gnosis_web3 = Web3(Web3.HTTPProvider(os.environ["GNOSISNODEURL"]))
poly_web3 = Web3(Web3.HTTPProvider("https://polygon-rpc.com"))
poly_web3.middleware_onion.inject(geth_poa_middleware, layer=0)

# ARBITRUM
# arb_block_now = arb_web3.eth.block_number - 1000
timestamp_now = 1694790000
arb_block = get_block_by_ts(timestamp_now, "arbitrum")  # 18 August 2023
# Given Arb block time, we want to look back 2 weeks:
arb_timestamp_2_weeks_ago = timestamp_now - (2 * 7 * 24 * 60 * 60)
arb_block_2_weeks_ago = get_block_by_ts(arb_timestamp_2_weeks_ago, "arbitrum")
# Convert to datetime:
arb_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
arb_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(arb_timestamp_2_weeks_ago)
# MAINNET
mainnet_block = get_block_by_ts(timestamp_now, "mainnet")
# Given mainnet block time, we want to look back 2 weeks:
mainnet_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
mainnet_block_2_weeks_ago = get_block_by_ts(mainnet_timestamp_2_weeks_ago, "mainnet")
# Convert to datetime:
mainnet_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
mainnet_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(mainnet_timestamp_2_weeks_ago)

# BASE
base_block = get_block_by_ts(timestamp_now, "base")
# Given base block time, we want to look back 2 weeks:
base_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
base_block_2_weeks_ago = get_block_by_ts(base_timestamp_2_weeks_ago, "base")
# Convert to datetime:
base_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
base_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(base_timestamp_2_weeks_ago)

# POLYGON
poly_block = get_block_by_ts(timestamp_now, "polygon")
# Given polygon block time, we want to look back 2 weeks:
poly_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
poly_block_2_weeks_ago = get_block_by_ts(poly_timestamp_2_weeks_ago, "polygon")
# Convert to datetime:
poly_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
poly_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(poly_timestamp_2_weeks_ago)

# GNOSIS
gnosis_block = get_block_by_ts(timestamp_now, "gnosis")
# Given gnosis block time, we want to look back 2 weeks:
gnosis_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
gnosis_block_2_weeks_ago = get_block_by_ts(gnosis_timestamp_2_weeks_ago, "gnosis")
# Convert to datetime:
gnosis_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
gnosis_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(gnosis_timestamp_2_weeks_ago)

print("Collecting bpt prices for Base")
base_bpt_twap_prices = {}
for base_pool in BASE_CORE_POOLS:
    base_bpt_twap_prices[base_pool] = get_twap_bpt_price(base_pool, 'base', base_web3,
                                                         start_date=base_datetime_now, block_number=base_block)

print("Collecting bpt prices for Gnosis")
gnosis_bpt_twap_prices = {}
for gnosis_pool in GNOSIS_CORE_POOLS:
    gnosis_bpt_twap_prices[gnosis_pool] = get_twap_bpt_price(gnosis_pool, 'gnosis', gnosis_web3,
                                                             start_date=gnosis_datetime_now,
                                                             block_number=gnosis_block)
print("Collecting bpt prices for Arbitrum")
arb_bpt_twap_prices = {}
for arb_pool in ARB_CORE_POOLS:
    arb_bpt_twap_prices[arb_pool] = get_twap_bpt_price(arb_pool, 'arbitrum', arb_web3,
                                                       start_date=arb_datetime_now,
                                                       block_number=arb_block)

print("Collecting bpt prices for Mainnet")
mainnet_bpt_twap_prices = {}
for mainnet_pool in MAINNET_CORE_POOLS:
    mainnet_bpt_twap_prices[mainnet_pool] = get_twap_bpt_price(mainnet_pool, 'mainnet', eth_web3,
                                                               start_date=mainnet_datetime_now,
                                                               block_number=mainnet_block)
print("Collecting bpt prices for Polygon")
polygon_bpt_twap_prices = {}
for polygon_pool in POLYGON_CORE_POOLS:
    polygon_bpt_twap_prices[polygon_pool] = get_twap_bpt_price(polygon_pool, 'polygon', poly_web3,
                                                               start_date=poly_datetime_now,
                                                               block_number=poly_block)

# Convert to dataframe and print, merge all three dataframes
arb_bpt_twap_prices_df = pd.DataFrame.from_dict(arb_bpt_twap_prices, orient='index')
mainnet_bpt_twap_prices_df = pd.DataFrame.from_dict(mainnet_bpt_twap_prices, orient='index')
polygon_bpt_twap_prices_df = pd.DataFrame.from_dict(polygon_bpt_twap_prices, orient='index')
base_bpt_twap_prices_df = pd.DataFrame.from_dict(base_bpt_twap_prices, orient='index')
gnosis_bpt_twap_prices_df = pd.DataFrame.from_dict(gnosis_bpt_twap_prices, orient='index')

In [2]:
# Arbitrum TWAP prices:
arb_bpt_twap_prices_df

,0
0xade4a71bb62bec25154cfc7e6ff49a513b491e81000000000000000000000497,1623.803750208670544721531711
0x9791d590788598535278552eecd4b211bfc790cb000000000000000000000498,1624.377167145606542517572864
0x4a2f6ae7f3e5d715689530873ec35593dc28951b000000000000000000000481,1628.349990901255710681940173
0x423a1323c871abc9d89eb06855bf5347048fc4a5000000000000000000000496,1.000290411622935290988723035
0x32df62dc3aed2cd6224193052ce665dc181658410002000000000000000003bd,1.085121361272666366722232096
0x0c8972437a38b389ec83d1e666b69b8a4fcf8bfd00000000000000000000049e,1623.055472360201217228037355


In [3]:
# Mainnet TWAP prices:
mainnet_bpt_twap_prices_df

,0
0x93d199263632a4ef4bb438f1feb99e57b4b5f0bd0000000000000000000005c2,1623.704233697886625688396428
0x1e19cf2d73a72ef1332c882f20534b6519be0276000200000000000000000112,1664.053061602441306949969912
0xe7e2c68d3b13d905bbb636709cf4dfd21076b9d20000000000000000000005ca,1608.399720145805893958560146
0xf7a826d47c8e02835d94fb0aa40f0cc9505cb1340002000000000000000005e0,1622.925345307640519772727559
0xf16aee6a71af1a9bc8f56975a4c2705ca7a782bc0002000000000000000004bb,24.85644392155588581935152415
0xb08885e6026bab4333a80024ec25a1a3e1ff2b8a000200000000000000000445,1636.734888119542278716465017
0x8353157092ed8be69a9df8f95af097bbf33cb2af0000000000000000000005d9,0.9830630976459687168553009454
0xdfe6e7e18f6cc65fa13c8d8966013d4fda74b6ba000000000000000000000558,1632.067794968628228080534514
0x5f1f4e50ba51d723f12385a8a9606afc3a0555f5000200000000000000000465,52.99384449470139378302670902
0x1ee442b5326009bb18f2f472d3e0061513d1a0ff000200000000000000000464,59.12791936802349983216312938


In [4]:
# Polygon TWAP prices:
polygon_bpt_twap_prices_df

,0
0xf0ad209e2e969eaaa8c882aac71f02d8a047d5c2000200000000000000000b49,0.5401322782587776219051044593
0xee278d943584dd8640eaf4cc6c7a5c80c0073e85000200000000000000000bc7,0.5379829134501508902329096653
0x89b753153678bc434c610b7e9182297cada8ff29000000000000000000000c21,0.5356126500917254196941948882
0xcd78a20c597e367a4e478a2411ceb790604d7c8f000000000000000000000c22,0.5356715280318930511148908154
0xdc31233e09f3bf5bfe5c10da2014677c23b6894c000000000000000000000c23,1622.879821270877598181639411


In [5]:
# Base TWAP prices:
base_bpt_twap_prices_df

,0
0xfb4c2e6e6e27b5b4a07a36360c89ede29bb3c9b6000000000000000000000026,1623.464844744910192597795771
0xc771c1a5905420daec317b154eb13e4198ba97d0000000000000000000000023,1625.496140456140259308739761
0x0c659734f1eef9c63b7ebdf78a164cdd745586db000000000000000000000046,1.000338450165294782406314561


In [6]:
# Gnosis TWAP prices:
gnosis_bpt_twap_prices_df

,0
0xbad20c15a773bf03ab973302f61fabcea5101f0a000000000000000000000034,1631.279000754427652711922303


In [7]:
# Query:
POOLS_SNAPSHOTS_QUERY = """
{{
  poolSnapshots(
    first: {first}
    skip: {skip}
    orderBy: timestamp
    orderDirection: desc
    block: {{ number: {block} }}
    where: {{ protocolFee_not: null }}
  ) {{
    pool {{
      address
      id
      symbol
      totalProtocolFeePaidInBPT
      tokens {{
        symbol
        address
        paidProtocolFees
      }}
    }}
    timestamp
    protocolFee
    swapFees
    swapVolume
    liquidity
  }}
}}
"""

## Fetching data from the Balancer subgraphs

In [8]:
from notebooks.constants import BASE_BALANCER_GRAPH_URL
from notebooks.constants import POLYGON_BALANCER_GRAPH_URL
from notebooks.constants import ARB_BALANCER_GRAPH_URL
from notebooks.constants import MAINNET_BALANCER_GRAPH_URL
from notebooks.constants import GNOSIS_BALANCER_GRAPH_URL
from typing import Dict
from typing import List
from typing import Optional

from gql import Client
from gql import gql
from gql.transport.requests import RequestsHTTPTransport


# Fetch all the data from the balancer subgraph
def make_gql_client(url: str) -> Optional[Client]:
    transport = RequestsHTTPTransport(url=url, retries=3)
    return Client(
        transport=transport, fetch_schema_from_transport=True, execute_timeout=60
    )


def get_balancer_pool_snapshots(block: int, graph_url: str) -> Optional[List[Dict]]:
    client = make_gql_client(graph_url)
    all_pools = []
    limit = 1000
    offset = 0
    while True:
        result = client.execute(
            gql(POOLS_SNAPSHOTS_QUERY.format(first=limit, skip=offset, block=block)))
        all_pools.extend(result['poolSnapshots'])
        offset += limit
        if offset >= 5000:
            break
        if len(result['poolSnapshots']) < limit - 1:
            break
    return all_pools


arbi_pool_snapshots_now = get_balancer_pool_snapshots(arb_block, ARB_BALANCER_GRAPH_URL)
arbi_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(arb_block_2_weeks_ago), ARB_BALANCER_GRAPH_URL)

mainnet_pool_snapshots_now = get_balancer_pool_snapshots(mainnet_block, MAINNET_BALANCER_GRAPH_URL)
mainnet_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(mainnet_block_2_weeks_ago),
                                                                 MAINNET_BALANCER_GRAPH_URL)

polygon_pool_snapshots_now = get_balancer_pool_snapshots(poly_block, POLYGON_BALANCER_GRAPH_URL)
polygon_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(poly_block_2_weeks_ago),
                                                                 POLYGON_BALANCER_GRAPH_URL)

base_pool_snapshots_now = get_balancer_pool_snapshots(base_block, BASE_BALANCER_GRAPH_URL)
base_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(base_block_2_weeks_ago),
                                                              BASE_BALANCER_GRAPH_URL)

gnosis_pool_snapshots_now = get_balancer_pool_snapshots(gnosis_block, GNOSIS_BALANCER_GRAPH_URL)
gnosis_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(gnosis_block_2_weeks_ago),
                                                                GNOSIS_BALANCER_GRAPH_URL)

## Extract fee data for CORE pools:


In [9]:
from collections import defaultdict
from decimal import Decimal
from notebooks import fetch_token_price_balgql


def collect_fee_info(pools: list[str], chain: str, pools_now: list[dict], pools_shifted: list[dict],
                     start_date: datetime.datetime) -> tuple[dict, dict]:
    # Iterate through snapshots now and 2 weeks ago and extract fee data, by subtracting today's fee data from 2 weeks ago
    # and then summing across all pools
    fees = {}
    token_fees = defaultdict(list)
    for pool in pools:
        current_fees_snapshots = [x for x in pools_now if x['pool']['id'] == pool]
        current_fees_snapshots.sort(key=lambda x: x['timestamp'], reverse=True)
        fees_2_weeks_ago = [x for x in pools_shifted if x['pool']['id'] == pool]
        fees_2_weeks_ago.sort(key=lambda x: x['timestamp'], reverse=True)
        # Take first element of list, which is the most recent snapshot
        if not current_fees_snapshots or not fees_2_weeks_ago:
            continue
        pool_snapshot_now = current_fees_snapshots[0]
        pool_snapshot_2_weeks_ago = fees_2_weeks_ago[0]
        # Calculate fees
        pool_fee = float(pool_snapshot_now['protocolFee']) - float(pool_snapshot_2_weeks_ago['protocolFee'])
        pool_swap_fee = float(pool_snapshot_now['swapFees']) - float(pool_snapshot_2_weeks_ago['swapFees'])
        # Now we need to collect token fee info. Let's start with BPT tokens, which is Balancer pool token. Notice,
        # That totalProtocolFeePaidInBPT can be null, so we need to check for that
        bpt_token_fee = 0
        bpt_price_usd = arb_bpt_twap_prices[pool] if chain == 'arbitrum' else mainnet_bpt_twap_prices[
            pool] if chain == 'mainnet' else polygon_bpt_twap_prices[pool] if chain == 'polygon' else \
            base_bpt_twap_prices[pool] if chain == 'base' else gnosis_bpt_twap_prices[pool]
        if bpt_price_usd is None:
            bpt_price_usd = 0
        if pool_snapshot_now['pool']['totalProtocolFeePaidInBPT'] is not None and pool_snapshot_2_weeks_ago['pool'][
            'totalProtocolFeePaidInBPT'] is not None:
            bpt_token_fee = float(pool_snapshot_now['pool']['totalProtocolFeePaidInBPT']) - float(
                pool_snapshot_2_weeks_ago['pool']['totalProtocolFeePaidInBPT'])
            token_fees[pool_snapshot_now['pool']['symbol']].append({
                'token': pool_snapshot_now['pool']['symbol'],
                'token_fee': bpt_token_fee,
                'token_price': bpt_price_usd,
                'token_fee_in_usd': Decimal(bpt_token_fee) * bpt_price_usd,
                'token_addr': pool_snapshot_now['pool']['address'],
                'time_from': arb_datetime_2_weeks_ago,
                'time_to': arb_datetime_now,
                'chain': chain,
            })
        # Now collect fee info about fees paid in pool tokens. Pool tokens fee info is in pool.tokens dictionary. This will be separate dictionary
        else:
            bpt_price_usd = 0
            for token_data in pool_snapshot_now['pool']['tokens']:
                token_data_2_weeks_ago = \
                    [t for t in pool_snapshot_2_weeks_ago['pool']['tokens'] if t['address'] == token_data['address']][0]
                token_fee = float(token_data.get('paidProtocolFees', None)) - float(
                    token_data_2_weeks_ago.get('paidProtocolFees', None))
                # Get twap token price from CoinGecko
                token_price = fetch_token_price_balgql(token_data['address'], chain, start_date) or 0
                token_fees[pool_snapshot_now['pool']['symbol']].append({
                    'token': token_data['symbol'],
                    'token_fee': token_fee,
                    'token_price': token_price,
                    'token_fee_in_usd': Decimal(token_fee) * token_price if token_price is not None else 0,
                    'token_addr': token_data['address'],
                    'time_from': arb_datetime_2_weeks_ago,
                    'time_to': arb_datetime_now,
                    'chain': chain,
                })
        # Calculate non-BPT fees in USD
        fees[pool_snapshot_now['pool']['symbol']] = {
            'pool_fee': round(pool_fee, 2),
            'swap_fee': round(pool_swap_fee, 2),
            'bpt_token_fee': round(bpt_token_fee, 2),
            # Get fee in USD by multiplying bpt_token_fee by price of BPT token taken from twap_bpt_price
            'bpt_token_fee_in_usd': round(Decimal(bpt_token_fee) * bpt_price_usd, 2),
            'token_fees_in_usd': round(sum([x['token_fee_in_usd'] for x in
                                            token_fees[
                                                pool_snapshot_now['pool']['symbol']]]) if bpt_price_usd == 0 else 0, 2),
            'time_from': arb_datetime_2_weeks_ago,
            'time_to': arb_datetime_now,
            'chain': chain,
            'token_fees': token_fees[pool_snapshot_now['pool']['symbol']],
        }
    return fees


arb_fees = collect_fee_info(ARB_CORE_POOLS, 'arbitrum', arbi_pool_snapshots_now,
                            arbi_pool_snapshots_2_weeks_ago, arb_datetime_now)
mainnet_fees = collect_fee_info(MAINNET_CORE_POOLS, 'mainnet', mainnet_pool_snapshots_now,
                                mainnet_pool_snapshots_2_weeks_ago, mainnet_datetime_now)
polygon_fees = collect_fee_info(POLYGON_CORE_POOLS, 'polygon', polygon_pool_snapshots_now,
                                polygon_pool_snapshots_2_weeks_ago, poly_datetime_now)
base_fees = collect_fee_info(BASE_CORE_POOLS, 'base', base_pool_snapshots_now, base_pool_snapshots_2_weeks_ago,
                             base_datetime_now)
gnosis_fees = collect_fee_info(GNOSIS_CORE_POOLS, 'gnosis', gnosis_pool_snapshots_now,
                               gnosis_pool_snapshots_2_weeks_ago, gnosis_datetime_now)
# Convert to dataframe, sort by chain and pool fee
joint_fees = {**arb_fees, **mainnet_fees, **polygon_fees, **base_fees, **gnosis_fees}
joint_fees_df = pd.DataFrame.from_dict(joint_fees, orient='index')

In [10]:
# Remove `token_fees` field from dataframe, as it's too big
joint_fees_df_copy = joint_fees_df.drop(columns=['token_fees'])
# Display all rows in dataframe
pd.set_option('display.max_rows', 1000)
joint_fees_df_copy.sort_values(by=['chain', 'pool_fee'], ascending=False)

,pool_fee,swap_fee,bpt_token_fee,bpt_token_fee_in_usd,token_fees_in_usd,time_from,time_to,chain
ECLP-WMATIC-stMATIC,1471.99,249.38,2730.95,1475.07,0,2023-09-01 18:00:00,2023-09-15 18:00:00,polygon
maticX-WMATIC-BPT,1450.10,323.64,2724.34,1459.35,0,2023-09-01 18:00:00,2023-09-15 18:00:00,polygon
ECLP-WMATIC-MATICX,403.20,71.44,755.02,406.19,0,2023-09-01 18:00:00,2023-09-15 18:00:00,polygon
stMATIC-WMATIC-BPT,2.58,1.34,4.62,2.47,0,2023-09-01 18:00:00,2023-09-15 18:00:00,polygon
wstETH-WETH-BPT,0.00,0.00,0.00,0.00,0.00,2023-09-01 18:00:00,2023-09-15 18:00:00,polygon
B-rETH-STABLE,15892.61,2294.22,0.00,0.00,15797.59,2023-09-01 18:00:00,2023-09-15 18:00:00,mainnet
50rETH-50BADGER,9939.15,2894.08,166.20,9827.34,0,2023-09-01 18:00:00,2023-09-15 18:00:00,mainnet
swETH-WETH-BPT,5502.29,634.17,3.43,5510.70,0,2023-09-01 18:00:00,2023-09-15 18:00:00,mainnet
ankrETH/wstETH,3867.44,126.65,2.32,3789.39,0,2023-09-01 18:00:00,2023-09-15 18:00:00,mainnet
20WETH-80ALCX,2393.07,3925.29,83.27,2069.92,0,2023-09-01 18:00:00,2023-09-15 18:00:00,mainnet


## Now let's calculate bribes paid to the pools

In [11]:
from notebooks import calculate_aura_vebal_share

aura_vebal_share = calculate_aura_vebal_share(eth_web3, mainnet_block)

# Bribes are split per chain and per pool, with each pool getting a share of the bribe proportional to its share of fees
# paid by all pools on that chain. For example, if pool A paid 10% of all fees on Arbitrum, it will get 10% of the bribes. That 10% will be distributed between aura and vebal, proportional to their share of the bribe.
FEE = Decimal(0.5)  # 50% goes to fees


# Let's calculate share of fees paid by each pool on each chain
def calc_and_split_bribes(fees: Dict, chain: str) -> Dict[str, Dict]:
    pool_bribs = {}
    for pool, data in fees.items():
        pool_fees = data['bpt_token_fee_in_usd'] + data['token_fees_in_usd']
        # Split fees between aura and bal fees
        pool_bribs[pool] = {
            "chain": chain,
            "aura_bribes": pool_fees * FEE * aura_vebal_share,
            "bal_bribes": pool_fees * FEE * (1 - aura_vebal_share),
            "pool_total": pool_fees,
            "fees_taken": pool_fees * FEE,
        }
    return pool_bribs


mainnet_bribes = calc_and_split_bribes(mainnet_fees, 'mainnet')
arb_bribes = calc_and_split_bribes(arb_fees, 'arbitrum')
polygon_bribes = calc_and_split_bribes(polygon_fees, 'polygon')
base_bribes = calc_and_split_bribes(base_fees, 'base')
gnosis_bribes = calc_and_split_bribes(gnosis_fees, 'gnosis')
# Convert to dataframe
joint_bribes_data = {**arb_bribes, **mainnet_bribes, **polygon_bribes, **base_bribes, **gnosis_bribes}
# Sort by chain:
joint_bribes_data = {k: v for k, v in sorted(joint_bribes_data.items(), key=lambda item: item[1]['chain'])}
joint_bribes_df = pd.DataFrame.from_dict(joint_bribes_data, orient='index')
# Sort by chain
# Dump into csv and prefix with dates
joint_bribes_df.to_csv(f'../data/bribs/bribes_{arb_datetime_2_weeks_ago.date()}_{arb_datetime_now.date()}.csv')
joint_bribes_df

,chain,aura_bribes,bal_bribes,pool_total,fees_taken
wstETH/rETH/cbETH,arbitrum,393.8127130999731453917867221,596.4622869000268546082132779,1980.55,990.275
4POOL-BPT,arbitrum,93.67952514341488373857387007,141.8854748565851162614261299,471.13,235.565
RDNT-WETH,arbitrum,2257.320035699507275590889023,3418.899964300492724409110977,11352.44,5676.220
wstETH/rETH/sfrxETH,arbitrum,0.003976801525838510973131571756,0.006023198474161489026868428244,0.02,0.010
rETH-WETH-BPT,base,0E-31,0E-31,0.00,0.000
cbETH/WETH,base,367.3789133577245629533811646,556.4260866422754370466188354,1847.61,923.805
bb-WETH-wstETH,gnosis,614.6445018297856597297829017,930.9304981702143402702170983,3091.15,1545.575
B-rETH-STABLE,mainnet,3141.194000828560128201679333,4757.600999171439871798320667,15797.59,7898.795
swETH-WETH-BPT,mainnet,1095.748008421914120981807624,1659.601991578085879018192376,5510.70,2755.350
ECLP-wstETH-cbETH,mainnet,0E-31,0E-31,0.00,0.000
